In [1]:
import math
import numpy as np

In [2]:
EPS = 0.1
ITERATIONS = 10000

In [3]:
def _format_matrix(min, max, size):
    return (max - min) * np.random.random((size, size)) + min
    
def rand_matrix(size):
    return np.random.random((size, size))

def diag_matrix(size):
    A = _format_matrix(0.0, 10.0, size)
    for i in range(size):
        A[i, i] = sum(A[i]) + 1
    return A

def hilbert_matrix(size):
    ans = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            ans[i, j] = 1.0 / (i + j + 1)
    return ans

def gen_right_column(size):
    return np.random.random(size)

In [17]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [21]:
def zeidelMethod(A, b):
    n = len(A)
    x = [0.0 for i in range(n)]
    xn = [1000000 for i in range(n)]
    
    curIterations = 0
    
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        xn = np.copy(x)
        curIterations += 1
        
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
    
    return xn

In [22]:
def _test(method, matrix, column):
    return method(matrix, column)

def _testAll(matrix_gen, size, methods):
    matrix, column = matrix_gen(size), gen_right_column(size)
    for method in methods:
        print(_test(method, matrix, column))

def _get_methods():
    return [zeidelMethod]

def testDiagMatrix(size):
    _testAll(diag_matrix, size, _get_methods())

def testHilbertMatrix(size):
    _testAll(hilbert_matrix, size, _get_methods())
    
def testRandMatrix(size):
    _testAll(rand_matrix, size, _get_methods())

In [23]:
testDiagMatrix(10)

[ 1.21714471e-02  1.24248916e-02  1.13043720e-02  3.60389744e-04
  1.86695475e-02  1.05357427e-02  4.16228863e-03 -4.95841670e-03
  3.75843957e-03  1.65759538e-07]


In [24]:
testRandMatrix(10)

[  1.11723328  -0.13501562   0.65241429  -0.24341142  12.2473532
 -19.53947457   2.78191716  18.68562984  -8.20308127   3.15466103]


In [25]:
testHilbertMatrix(10)

[ 0.49008127 -0.28742758  3.54151789 -3.12171602 -0.65263511  4.65422826
  3.75532995 -8.42994646 -0.77472257  4.77446518]
